In [1]:
#We will be doing cats and dogs clustering without sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [2]:
#importing the data
data_dir = '/home/kalyan/DataSets/DogsandCats/'
train_dir = os.path.join(data_dir, 'training_set/training_set/')
test_dir = os.path.join(data_dir, 'test_set/test_set')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
test_cats_dir = os.path.join(test_dir, 'cats')
test_dogs_dir = os.path.join(test_dir, 'dogs')

#checking the number of images in each folder
print('Total training cat images:', len(os.listdir(train_cats_dir)))
print('Total training dog images:', len(os.listdir(train_dogs_dir)))
print('Total test cat images:', len(os.listdir(test_cats_dir)))
print('Total test dog images:', len(os.listdir(test_dogs_dir)))


Total training cat images: 4000
Total training dog images: 4000
Total test cat images: 1000
Total test dog images: 1000


In [3]:
#taking svd of the images and to truncatate first 10 features of the images as it is most contributing to the images

def svd_truncate(img, n):
    #converting images to grayscale
    img = img.mean(axis=2)
    U, s, V = np.linalg.svd(img)
    U = U[:, :n]
    s = s[:n]
    us = np.dot(U, np.diag(s))
    return us

In [4]:
#getting svd of all images
def get_svd_images(img_dir, n):
    img_files = os.listdir(img_dir)
    img_files = [os.path.join(img_dir, f) for f in img_files]
    img_files = [plt.imread(f) for f in img_files]
    svd_images = [svd_truncate(img, n) for img in img_files]
    return svd_images


In [5]:
images = get_svd_images(train_cats_dir, 10)
images = np.array(images)
images = images.reshape(images.shape[0], -1)
print(images.shape)

(4000, 1)


/tmp/ipykernel_53563/26831160.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  images = np.array(images)


In [6]:
cat_images = images

In [7]:
dog_images = get_svd_images(train_dogs_dir, 10)
dog_images = np.array(dog_images)
dog_images = dog_images.reshape(dog_images.shape[0], -1)
print(dog_images.shape)


(4000, 1)


/tmp/ipykernel_53563/342760469.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  dog_images = np.array(dog_images)


In [8]:
#concatenating the images
images = np.concatenate((cat_images, dog_images), axis=0)
print(images.shape)

(8000, 1)


In [18]:
#creating labels for the images
labels = np.concatenate((np.zeros(cat_images.shape[0]), np.ones(dog_images.shape[0])), axis=0)
print(labels.shape)

(8000,)


In [24]:
#to solve this only size-1 arrays can be converted to Python scalars
labels = labels.astype(int)
print(labels.shape)

(8000,)


In [25]:
#creating a dataframe
df = pd.DataFrame(images)
df['label'] = labels
df.head()

,0,label
0,"[[-1199.6142400105557, 0.07755592133244017, -4...",0
1,"[[-4817.572329818299, 1517.1136197768647, 45.8...",0
2,"[[-2049.788186463003, -48.07080475123446, -58....",0
3,"[[-1304.2628575741167, 278.0126601613926, 266....",0
4,"[[-111.5193419620896, 38.90981610400313, 52.97...",0


In [ ]:
#doing same for test images
test_images = get_svd_images(test_cats_dir, 10)
test_images = np.array(test_images)
test_images = test_images.reshape(test_images.shape[0], -1)
print(test_images.shape)


(1000, 1)


/tmp/ipykernel_53563/897506422.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_images = np.array(test_images)


In [12]:
test_cat_images = test_images

In [13]:
#test for dogs
test_dog_images = get_svd_images(test_dogs_dir, 10)
test_dog_images = np.array(test_dog_images)
test_dog_images = test_dog_images.reshape(test_dog_images.shape[0], -1)
print(test_dog_images.shape)


(1000, 1)


/tmp/ipykernel_53563/2417915150.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_dog_images = np.array(test_dog_images)


In [14]:
#concatenating the test images
test_images = np.concatenate((test_cat_images, test_dog_images), axis=0)
print(test_images.shape)

(2000, 1)


In [1]:
#model building
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


X_train = df.drop('label', axis=1)
y_train = df['label']
X_test = test_images
y_test = np.concatenate((np.zeros(test_cat_images.shape[0]), np.ones(test_dog_images.shape[0])), axis=0)
y_test = y_test.astype(int)


NameError: name 'df' is not defined

In [27]:
#knn model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
print('Accuracy of KNN model is:', accuracy_score(y_test, y_pred))

ValueError: setting an array element with a sequence.